# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [29]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import re
import nltk
import pickle
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score,make_scorer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from xgboost import XGBClassifier

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ivankhokhlov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ivankhokhlov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ivankhokhlov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/ivankhokhlov/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [30]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table(table_name="disaster_messages", con=engine)
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [31]:
df.isna().sum()

id                            0
message                       0
original                  16046
genre                         0
related                       0
request                       0
offer                         0
aid_related                   0
medical_help                  0
medical_products              0
search_and_rescue             0
security                      0
military                      0
child_alone                   0
water                         0
food                          0
shelter                       0
clothing                      0
money                         0
missing_people                0
refugees                      0
death                         0
other_aid                     0
infrastructure_related        0
transport                     0
buildings                     0
electricity                   0
tools                         0
hospitals                     0
shops                         0
aid_centers                   0
other_in

In [32]:
Y = df.drop(columns=['message', 'original', 'genre', 'id'])
X = df['message']
column_names = Y.columns


pandas.core.indexes.base.Index

### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()).strip()
    
    # Todo: tokenize text
    words = word_tokenize(text)
    
    # Todo: lemmatize and remove stop words
    tokens = [w for w in words if w not in stop_words]
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in tokens]
    return lemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(XGBClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, train_size=0.8, random_state=88)

In [12]:
pipeline.fit(xtrain, ytrain)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x1374752d0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                               booster=None,
                                                               callbacks=None,
                                                               colsample_bylevel=None,
                                                               colsample_bynode=None,
                                                               colsample_bytree=None,
                                                               early_stopping_rounds=None,
                                                               enable_categorical=False,
                                                               eval_metric=None,
                                                               gamm...
                                                               grow_policy=None,
                                                               importance_type=None,
                                                               interaction_constraints=None,
                                                               learning_rate=None,
                                                               max_bin=None,
                                                               max_cat_to_onehot=None,
                                                               max_delta_step=None,
                                                               max_depth=None,
                                                               max_leaves=None,
                                                               min_child_weight=None,
                                                               missing=nan,
                                                               monotone_constraints=None,
                                                               n_estimators=100,
                                                               n_jobs=None,
                                                               num_parallel_tree=None,
                                                               predictor=None,
                                                               random_state=None,
                                                               reg_alpha=None,
                                                               reg_lambda=None, ...)))])

In [13]:
ypred = pipeline.predict(xtest)

In [14]:
ypred = pd.DataFrame(ypred)

In [15]:
xtest.head()

21187    About 12,000 patients have been airlifted out ...
6887     I would like to know when the opening of the s...
10944         Canned foods : veggies , potatoes , sauces .
764      Please come and help us we are in pis Aviation...
2952     He is tying to find out if there is a water di...
Name: message, dtype: object

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
ypred.shape

(5244, 36)

In [17]:
ytest.shape

(5244, 36)

In [18]:
ypred.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
ytest.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
21187,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6887,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10944,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
764,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2952,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
def get_score(ytest, ypred, column_names):
    results = []
    for i, column_name in enumerate(column_names):
        accuracy = accuracy_score(ytest.iloc[:, i], ypred.iloc[:, i])
        f1 = f1_score(ytest.iloc[:, i], ypred.iloc[:, i], average='micro')
        precision = precision_score(ytest.iloc[:, i], ypred.iloc[:, i], average='micro')
        recall = recall_score(ytest.iloc[:, i], ypred.iloc[:, i], average='micro')        
        results.append([column_name, accuracy, f1, precision, recall])
    return pd.DataFrame(results, columns=['category', 'accuracy score', 'f1 score', 'precision score', 'recall score'])

In [21]:
scores = get_score(ytest, ypred, column_names)
scores.sort_values(by=["f1 score"]).head(50)

,category,accuracy score,f1 score,precision score,recall score
3,aid_related,0.776888,0.776888,0.776888,0.776888
0,related,0.812166,0.812166,0.812166,0.812166
35,direct_report,0.862128,0.862128,0.862128,0.862128
18,other_aid,0.873951,0.873951,0.873951,0.873951
28,weather_related,0.884821,0.884821,0.884821,0.884821
1,request,0.902555,0.902555,0.902555,0.902555
4,medical_help,0.924867,0.924867,0.924867,0.924867
19,infrastructure_related,0.929443,0.929443,0.929443,0.929443
34,other_weather,0.944317,0.944317,0.944317,0.944317
30,storm,0.944508,0.944508,0.944508,0.944508


### 6. Improve your model
Use grid search to find better parameters. 

In [247]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(XGBClassifier()))
    ])

In [ ]:
parameters = {
    'clf__estimator__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'clf__estimator__max_depth': [3, 5, 10, 15],
    'clf__estimator__min_child_weight': [0, 1 ,2]
}

In [ ]:
cv = RandomizedSearchCV(estimator=pipeline,
                        param_distributions=parameters,
                        n_iter=10,
                        n_jobs=-1,
                        verbose=100,
                        random_state=88
                       )

cv.fit(xtrain, ytrain)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [28]:
pipeline

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x1374752d0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                               booster=None,
                                                               callbacks=None,
                                                               colsample_bylevel=None,
                                                               colsample_bynode=None,
                                                               colsample_bytree=None,
                                                               early_stopping_rounds=None,
                                                               enable_categorical=False,
                                                               eval_metric=None,
                                                               gamm...
                                                               grow_policy=None,
                                                               importance_type=None,
                                                               interaction_constraints=None,
                                                               learning_rate=None,
                                                               max_bin=None,
                                                               max_cat_to_onehot=None,
                                                               max_delta_step=None,
                                                               max_depth=None,
                                                               max_leaves=None,
                                                               min_child_weight=None,
                                                               missing=nan,
                                                               monotone_constraints=None,
                                                               n_estimators=100,
                                                               n_jobs=None,
                                                               num_parallel_tree=None,
                                                               predictor=None,
                                                               random_state=None,
                                                               reg_alpha=None,
                                                               reg_lambda=None, ...)))])

In [23]:
pipeline.named_steps["clf"].estimator

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None,
              reg_alpha=None, reg_lambda=None, ...)

In [25]:
pipeline.named_steps["vect"]

CountVectorizer(tokenizer=<function tokenize at 0x1374752d0>)

In [27]:
outfile = open("pipeline.pkl", "wb")
pickle.dump(pipeline, outfile)
outfile.close()

CountVectorizer(tokenizer=<function tokenize at 0x1374752d0>)

In [26]:
model = pipeline.named_steps['vect']
outfile = open("vectorizer.pkl", "wb")
pickle.dump(model, outfile)
outfile.close()
model

CountVectorizer(tokenizer=<function tokenize at 0x1374752d0>)

In [24]:
model = pipeline.named_steps['clf'].estimator
outfile = open("xgb_model.pkl", "wb")
pickle.dump(model, outfile)
outfile.close()
model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None,
              reg_alpha=None, reg_lambda=None, ...)

In [ ]:
pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.